In [1]:
import re
import os
import pprint

какие теги есть в каждом документе, какие есть во всех

In [2]:
tags = dict()

for fn in os.listdir(r'C:\Users\mkuzo\Desktop\Корпус кпелле'):
    if fn.endswith('.txt'):
        with open(fn, 'r', encoding='utf8') as f:
            text = f.read()
            
        tags[fn[:4]] = set(re.findall(r'^(?:\\.+?)(?=\t)', text, re.MULTILINE))
tags

{'2018': {'\\comm', '\\ftf', '\\ge', '\\id', '\\mb', '\\ps', '\\srf'}}

In [3]:
#set.intersection(tags['2014'], tags['2018'], tags['kpel'])

In [4]:
tags2 = dict()

for fn in os.listdir(r'C:\Users\mkuzo\Desktop\Корпус кпелле'):
    if fn.endswith('.txt'):
        with open(fn, 'r', encoding='utf8') as f:
            text = f.read()
        
        text = re.sub(r'(?<=\n)\t+', '===========', text)
        sents = text.split('===========')
        curr_tags = set()
        for s in sents:
            curr_tags.add(tuple(set(re.findall(r'^(?:\\.+?)(?=\t)', s, re.MULTILINE))))
        
        tags2[fn[:4]] = curr_tags

for k, v in tags2.items():
    v_list = [set(_) for _ in v if len(_)>1]
    print(set.intersection(*v_list))
    print(len(v))
    
print(tags2)

{'\\id', '\\srf', '\\ftf', '\\ge', '\\ps', '\\mb'}
2
{'2018': {('\\id', '\\srf', '\\comm', '\\ftf', '\\ge', '\\ps', '\\mb'), ('\\id', '\\srf', '\\ftf', '\\ge', '\\ps', '\\mb')}}


корпус

In [5]:
class Doc:

    def __init__(self, uid, name, filename):
        self.uid = uid
        self.name = name
        self.file = filename

class Sentence:

    def __init__(self, uid, doc_id, pos, text, translation):
        self.uid = uid
        self.doc_id = doc_id
        self.text = text
        self.translation = translation
        self.pos = pos


class Token:

    def __init__(self, uid, sent_id, pos, text, ps):
        self.uid = uid
        self.sent_id = sent_id
        self.text = text
        self.ps = ps
        self.pos = pos
        
    def __str__(self):
        return str(vars(self))
        
class TokenToMorph:
    
    def __init__(self, word_id, morph_id, pos):
        self.word_id = word_id
        self.morph_id = morph_id
        self.pos = pos

class Morph:
    
    def __init__(self, uid, text, ge, ps):
        self.uid = uid
        self.full_text = text
        self.text = text.strip('-').strip('_')
        self.ge = ge
        self.ps = ps
        self.isroot = int('-' not in text)
    
    def __eq__(self, other):
        return self.full_text == other.full_text and self.ge == other.ge and self.ps == other.ps 

    def __hash__(self):
        return hash((self.full_text, self.ge, self.ps))

    def __str__(self):
        return str(vars(self))


In [6]:
def parse_sent(s):
    parsed = dict()
    
#    parsed['ftf'] = re.sub('\t', ' ', re.search(r'(?<=^\\ftf).+$', s).group(0))
    parsed['id'] = re.search(r'(?<=\\id\t)\d+', s.lower()).group(0)
    parsed['ftf'] = re.sub('\t', ' ', re.search(r'(?<=^\\ftf).+$', s.lower(), re.MULTILINE).group(0))
    parsed['s_text'] = re.sub('\t', ' ', re.search(r'(?<=^\\srf).+$', s.lower(), re.MULTILINE).group(0))
    
    parsed['tokens'] = []
    for t in re.search(r'(?<=^\\srf).+$', s.lower(), re.MULTILINE).group(0).split('\t'):
        if t != '':
            parsed['tokens'].extend(re.findall(r'[^./,…\\‘;?%]+|[./,…\\‘;?%]+', t))
    
    parsed['mb'] = [re.sub(r'[./,…\\‘;?%]+', '', i) for i in re.search(r'(?<=^\\mb).+$', s.lower(), re.MULTILINE).group(0).split('\t') if i != '']
    parsed['ge'] = [i for i in re.search(r'(?<=^\\ge).+$', s.lower(), re.MULTILINE).group(0).split('\t') if i != '']
    parsed['ps'] = [i for i in re.search(r'(?<=^\\ps).+$', s.lower(), re.MULTILINE).group(0).split('\t') if i != '']
    
    parsed['mb'] = [i for i in parsed['mb'] if i != '']
    
    return parsed

def find_morph_uid(m, morphs):
    for i in morphs:
        if m == i:
            return i.uid


In [7]:
with open('2018_FPhL_voyage_glossed+surface+tones_ gl MKuz.txt', 'r', encoding='utf8') as f:
    text = f.read()
        
text = re.sub(r'(?<=\n)\t+', '===========', text)
sents = text.split('===========')

найти все морфемы

In [8]:
morphs = set()

for s in sents:
    parsed = parse_sent(s)
    for i in range(len(parsed['mb'])):
        morphs.add(Morph(None, parsed['mb'][i], parsed['ge'][i], parsed['ps'][i]))
    

curr_morph_uid = 0
for m in morphs:
    m.uid = curr_morph_uid
    curr_morph_uid += 1

In [9]:
curr_doc_id = 0

doc = [Doc(curr_doc_id, '2018_FPhL_voyage+NOT_glossed+surface+tones', '2018_FPhL_voyage_glossed+surface+tones_ gl MKuz.txt')]

sentences = []
tokens = []
tok_to_morph = []


curr_token_uid = 0


for s in sents:
    parsed = parse_sent(s)
    sentences.append(Sentence(parsed['id'], curr_doc_id, parsed['id'], parsed['s_text'], parsed['ftf']))
    
    non_punkt_tokens = []
    
    for i in range(len(parsed['tokens'])):
        if not (re.search(r'[./,…\\‘;?%]', parsed['tokens'][i]) or parsed['tokens'][i] == '-'):
            non_punkt_tokens.append(Token(curr_token_uid, parsed['id'], i, parsed['tokens'][i], None))
        
        tokens.append(Token(curr_token_uid, parsed['id'], i, parsed['tokens'][i], 'punct'))
        
        curr_token_uid += 1
    
    morphs_id_groups = []
    prev_morphs = []
    for i in range(len(parsed['mb'])):
        curr_m = Morph(None, parsed['mb'][i], parsed['ge'][i], parsed['ps'][i])
        
#        print(*prev_morphs, '===', curr_m)
        
        if prev_morphs == [] or curr_m.full_text.startswith('-'):
            prev_morphs.append(curr_m)
        
        else:
#            morphs_id_groups.append(prev_morphs)
            morphs_id_groups.append([find_morph_uid(j, morphs) for j in prev_morphs])
            prev_morphs = [curr_m]
        
    morphs_id_groups.append([find_morph_uid(j, morphs) for j in prev_morphs])
            
    if len(non_punkt_tokens) != len(morphs_id_groups):
        print('____')
        print(*[m.text for m in non_punkt_tokens])
#        print(*[[m.text for m in x] for x in morphs_id_groups])
        continue
        
    for i in range(len(non_punkt_tokens)):
        for j in range(len(morphs_id_groups[i])):
            tok_to_morph.append(TokenToMorph(non_punkt_tokens[i].uid, morphs_id_groups[i][j], j))

____
élɛɛ láa zù hùtíli è kɛ́i kɛ̀â pú 19h lɔ̀wai
____
mìsílipɛ̀lɛi tíi ɓé kɛ̀â kú yíi mù
____
nàa ɓé kú kú kée kɔ́lɔŋ nàa depuis ɲèlei tíi kwá kɛ́ kú kée tólui kwá kɛ́ kú kée káa


делаем sql таблицу

In [10]:
import sqlite3

In [11]:
tables_to_write = {'Docs': doc, 'Sentences': sentences, 'Tokens': tokens, 'TokensToWords': tok_to_morph, 'Morphs': morphs}
columns = {'Docs': """id INTEGER, name TEXT, filename TEXT """, 
           'Sentences': """ id INTEGER, doc_id INTEGER, pos INTEGER, text TEXT, translation TEXT""", 
           'Tokens': """ id INTEGER, sent_id INTEGER, pos INTEGER, text TEXT, ps TEXT""", 
           'TokensToWords': """ word_id INTEGER, morph_id INTEGER, pos INTEGER """, 
           'Morphs': """ id INTEGER, full_text TEXT, text TEXT, ge TEXT, ps TEXT"""}


queries = {'Docs': [(i.uid, i.name, i.file) for i in doc], 
           'Sentences': [(i.uid, i.doc_id, i.pos, i.text, i.translation) for i in sentences], 
           'Tokens': [(i.uid, i.sent_id, i.pos, i.text, i.ps)for i in tokens], 
           'TokensToWords': [(i.word_id, i.morph_id, i.pos)for i in tok_to_morph], 
           'Morphs': [(i.uid, i.full_text, i.text, i.ge, i.ps)for i in morphs]}  



In [12]:
with sqlite3.connect('kpele_db.db') as db:
    cursor = db.cursor()
    
    for k in tables_to_write.keys():
        insert_k = []
        query = f""" CREATE TABLE IF NOT EXISTS {k}({columns[k]}) """
        cursor.execute(query)
        
    db.commit()

In [13]:
with sqlite3.connect('kpele_db.db') as db:
    cursor = db.cursor()
    
    for k in tables_to_write.keys():
        query = f""" INSERT INTO {k}  VALUES({','.join(['?']*len(queries[k][0]))}); """
        cursor.executemany(query, queries[k])
        print(k, 'yay')
    db.commit()

Docs yay
Sentences yay
Tokens yay
TokensToWords yay
Morphs yay


##### это для определения использованных символов, это не важно

In [14]:
with open('2018_FPhL_voyage_glossed+surface+tones_ gl MKuz.txt', 'r', encoding='utf8') as f:
    text = f.read()
        
token_strings = set()
for i in re.findall(r'^\\srf.+$', text, re.MULTILINE):
    token_strings.update(i.split('\t'))



In [15]:
syms = ''.join(list(token_strings))

In [16]:
print(set(re.findall(r'[^0-9a-zA-Z-]', syms)))

{"'", '̀', 'ɓ', '/', '̂', 'ŋ', 'ɲ', '…', '.', ';', '–', '̧', '\\', 'ɛ', '́', '̌', '?', 'ɣ', ',', 'ɔ', '‘', '%', '!', '’'}


In [17]:
re.finditer(r'a', 'sd')